# Review NLP Helpfulness Prediction PretrainedModels (Books)

## Creating Spark Session & Importing All Necessary Libraries

In [1]:
import findspark
findspark.init()

In [2]:
# Import Spark NLP
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql import functions as F

import sparknlp

# Start SparkSession with Spark NLP
# start() functions has 5 parameters: gpu, spark23, spark24, spark32, and memory
# sparknlp.start(gpu=True) will start the session with GPU support
# sparknlp.start(spark23=True) is when you have Apache Spark 2.3.x installed
# sparknlp.start(spark24=True) is when you have Apache Spark 2.4.x installed
# sparknlp.start(spark32=True) is when you have Apache Spark 3.2.x installed
# sparknlp.start(memory="16G") to change the default driver memory in SparkSession
spark = sparknlp.start(gpu = True, spark32= True)

### Reading in Data

In [3]:
df = spark.read \
    .option("quote", "\"")  \
    .option("escape", "\"") \
    .option("ignoreLeadingWhiteSpace",True) \
    .csv(r"C:\Users\kenne\OneDrive\Desktop\UChicago\Python\Final Project Big Data Amazon Review\Amazon Review Kaggle Data\amazon_reviews_us_Books_v1_02.tsv",inferSchema=True,header=True, sep='\t' )

## Code Cleaning

In [4]:
df.show(1, vertical = True, truncate = False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [5]:
df = df.select('product_id', 'star_rating', 'product_category', 'review_headline', 'review_body', 'helpful_votes', 'total_votes')

In [6]:
df.show(1, vertical = True, truncate = False)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
df.dropna().count()

3105463

In [8]:
from pyspark.sql.functions import col,isnan, when, count
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]
   ).show()

+----------+-----------+----------------+---------------+-----------+-------------+-----------+
|product_id|star_rating|product_category|review_headline|review_body|helpful_votes|total_votes|
+----------+-----------+----------------+---------------+-----------+-------------+-----------+
|         0|          5|               2|             32|         31|            5|          5|
+----------+-----------+----------------+---------------+-----------+-------------+-----------+



In [9]:
df2 = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])
df2.count()
df2.show(5, vertical = True)

-RECORD 0-----------------
 product_id       | 0     
 star_rating      | 5     
 product_category | 2     
 review_headline  | 487   
 review_body      | 15421 
 helpful_votes    | 5     
 total_votes      | 5     



In [10]:
df = df.filter(col('total_votes') > 10)
df.count()

960537

In [11]:
df = df.fillna("", "review_body")
df = df.fillna("", "review_headline")

In [12]:
df = df.withColumn('review_text', F.concat('review_headline', F.lit(" "), 'review_body'))
df.show(1, vertical = True, truncate = False)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------
 product_id       | 1844161560                                                                                                                   
 star_rating      | 4                                                                                                                            
 product_category | Books                                                                                                                        
 review_headline  | this isn't a review                                                                                                          
 review_body      | never read it-a young relative idicated he liked it and somehow my name popped upon this--no more to say                     
 helpful_votes    | 1                                                                                                       

In [13]:
df.filter((col("review_text").isNull()) | ( col("review_text") == "")).show(1, vertical = True, truncate = False)

(0 rows)



In [14]:
df.select([count(when((col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns]).show(5, vertical = True)

-RECORD 0---------------
 product_id       | 0   
 star_rating      | 0   
 product_category | 0   
 review_headline  | 6   
 review_body      | 5   
 helpful_votes    | 0   
 total_votes      | 0   
 review_text      | 0   



In [15]:
df = df.withColumn('helpful_ratio', F.col('helpful_votes') / F.col('total_votes'))
df.show(1, vertical = True, truncate = False)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------
 product_id       | 1844161560                                                                                                                   
 star_rating      | 4                                                                                                                            
 product_category | Books                                                                                                                        
 review_headline  | this isn't a review                                                                                                          
 review_body      | never read it-a young relative idicated he liked it and somehow my name popped upon this--no more to say                     
 helpful_votes    | 1                                                                                                       

In [16]:
df.filter(col('helpful_ratio') < 0).count()

0

In [17]:
df = df.withColumn('helpful', when(col("helpful_ratio") < 0.5, 0).otherwise(1))

In [18]:
df.show(5, vertical = True)

-RECORD 0--------------------------------
 product_id       | 1844161560           
 star_rating      | 4                    
 product_category | Books                
 review_headline  | this isn't a review  
 review_body      | never read it-a y... 
 helpful_votes    | 1                    
 total_votes      | 22                   
 review_text      | this isn't a revi... 
 helpful_ratio    | 0.045454545454545456 
 helpful          | 0                    
-RECORD 1--------------------------------
 product_id       | 0805076069           
 star_rating      | 4                    
 product_category | Books                
 review_headline  | I viewed this the... 
 review_body      | This book is chil... 
 helpful_votes    | 9                    
 total_votes      | 11                   
 review_text      | I viewed this the... 
 helpful_ratio    | 0.8181818181818182   
 helpful          | 1                    
-RECORD 2--------------------------------
 product_id       | 0300108834    

In [19]:
df = df.sample(False, 0.5, seed = 0)

## USE Pipeline

In [20]:
#USE (Universal Sentence Encoder) Sentence Embedding
document = DocumentAssembler()\
    .setInputCol("review_text")\
    .setOutputCol("document")
    
embeddingsSentence = UniversalSentenceEncoder.load('tfhub_use_en_2.4.0_2.4_1587136330099') \
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")

classifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("prediction")\
  .setLabelColumn("helpful")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

use_clf_pipeline = Pipeline(
    stages = [
        document,
        embeddingsSentence,
        classifierdl
    ])

In [24]:
train, test = df.randomSplit([0.8, 0.2], seed=12345)

In [25]:
%%time
pipeline2 = use_clf_pipeline.fit(train)

Wall time: 4min 38s


In [26]:
%%time
final = pipeline2.transform(test)

Wall time: 92.6 ms


In [27]:
metrics = final.select('helpful','product_id',"prediction.result")

## USE Results

In [28]:
%%time
from sklearn.metrics import classification_report, accuracy_score
metrics_final = metrics.toPandas()
metrics_final['result'] = metrics_final['result'].apply(lambda x: x[0])
metrics_final['result'] = metrics_final['result'].astype('int')

print(classification_report(metrics_final.helpful, metrics_final.result))
print(accuracy_score(metrics_final.helpful, metrics_final.result))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     16774
           1       0.83      1.00      0.90     79594

    accuracy                           0.83     96368
   macro avg       0.41      0.50      0.45     96368
weighted avg       0.68      0.83      0.75     96368

0.8259380707288727
Wall time: 20.9 s


C:\Users\kenne\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## USE Inference

In [29]:
light_model = LightPipeline(pipeline2)
#Using a review that was stated Helpful on Amazon
text="I’m only 19 pages in and will pick this up every available moment until I am finished. Well written and an excellent guide and research tool! I just wish it had arrived in pristine condition. The box was not damaged but the top right corner is crumpled!"
light_model.annotate(text)['prediction'][0]

'1'

In [30]:
#Using a review that has not beed stated Helpful on Amazon YET
text="Was good"
light_model.annotate(text)['prediction'][0]

'1'

In [31]:
#Using a review that has not beed stated Helpful on Amazon YET
text="No complaints"
light_model.annotate(text)['prediction'][0]

'1'

## BERT_uncased Embedding Pipeline

In [32]:
#BERT Sentence Embedding
document = DocumentAssembler()\
    .setInputCol("review_text")\
    .setOutputCol("document")
    
bert_cmlm = BertSentenceEmbeddings.load('sent_bert_base_uncased_en_2.6.0_2.4_1598346203624')\
.setInputCols(["document"])\
.setOutputCol("sentence_embeddings")

classifierdl = ClassifierDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("prediction")\
  .setLabelColumn("helpful")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

use_clf_pipeline = Pipeline(
    stages = [
        document,
        embeddingsSentence,
        classifierdl
    ])

In [33]:
train, test = df.randomSplit([0.8, 0.2], seed=12345)

In [34]:
%%time
pipeline2 = use_clf_pipeline.fit(train)

Wall time: 4min 32s


In [35]:
%%time
final = pipeline2.transform(test)

Wall time: 38 ms


In [36]:
metrics = final.select('helpful','product_id',"prediction.result")

## BERT_uncased Embedding Results

In [37]:
%%time
from sklearn.metrics import classification_report, accuracy_score
metrics_final = metrics.toPandas()
metrics_final['result'] = metrics_final['result'].apply(lambda x: x[0])
metrics_final['result'] = metrics_final['result'].astype('int')

print(classification_report(metrics_final.helpful, metrics_final.result))
print(accuracy_score(metrics_final.helpful, metrics_final.result))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00     16774
           1       0.83      1.00      0.90     79594

    accuracy                           0.83     96368
   macro avg       0.41      0.50      0.45     96368
weighted avg       0.68      0.83      0.75     96368

0.8259380707288727
Wall time: 24.3 s


## BERT Inference

In [38]:
light_model = LightPipeline(pipeline2)
#Using a review that was stated Helpful on Amazon
text="I’m only 19 pages in and will pick this up every available moment until I am finished. Well written and an excellent guide and research tool! I just wish it had arrived in pristine condition. The box was not damaged but the top right corner is crumpled!"
light_model.annotate(text)['prediction'][0]

'1'

In [39]:
#Using a review that has not beed stated Helpful on Amazon YET
text="Was Good"
light_model.annotate(text)['prediction'][0]

'1'

In [40]:
#Using a review that has not beed stated Helpful on Amazon YET
text="No complaints"
light_model.annotate(text)['prediction'][0]

'1'